In [8]:
from bs4 import BeautifulSoup
import requests
import scraperwiki

In [44]:
pages = ['http://www.aph.gov.au/Senators_and_Members/Parliamentarian_Search_Results?page=1&expand=1&q=&mem=1&par=-1&gen=0&ps=100&st=1',
        'http://www.aph.gov.au/Senators_and_Members/Parliamentarian_Search_Results?page=2&expand=1&q=&mem=1&par=-1&gen=0&ps=100&st=1']

basedata = []

for page in pages:
    print(page)
    soup = BeautifulSoup(requests.get(page).content)
    maindiv = soup.find('div',class_='search-filter-results')
    entries = maindiv.find_all('div',class_='row')
    for e in entries:
        row = {}
        dts = e.find_all('dt')
        dds = e.find_all('dd')
        row['name'] = e.find_all('a')[1].text
        row['link'] = e.find_all('a')[1].get('href')
        counter = 0
        while counter<len(dts):
            if dts[counter].text.strip() != 'Connect':
                row[dts[counter].text] = dds[counter].text
            counter+=1
        basedata.append(row)

http://www.aph.gov.au/Senators_and_Members/Parliamentarian_Search_Results?page=1&expand=1&q=&mem=1&par=-1&gen=0&ps=100&st=1


/Users/jacksongs/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/jacksongs/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


http://www.aph.gov.au/Senators_and_Members/Parliamentarian_Search_Results?page=2&expand=1&q=&mem=1&par=-1&gen=0&ps=100&st=1


In [51]:
offices = []
for b in basedata:
    page = requests.get('http://aph.gov.au'+b['link'])
    soup = BeautifulSoup(page.content)
    details = soup.find('div',id='panel21')
    address = details.find_all('p')
    for i,a in enumerate(address):
        if i % 2 == 0:
            b2 = b.copy()
            b2['address'] = a.text.strip()
            b2['office'] = i/2 + 1
            offices.append(b2)


/Users/jacksongs/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/jacksongs/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [54]:
scraperwiki.sqlite.save(unique_keys='',data=offices,table_name='offices')

AttributeError: module 'scraperwiki' has no attribute 'sqlite'

In [55]:
offices

[{'For': 'Warringah, New South Wales',
  'Party': 'Liberal Party of Australia',
  'address': 'Level 2, 17 Sydney Road\r\n                                             Manly, NSW, 2095',
  'link': '/Senators_and_Members/Parliamentarian?MPID=EZ5',
  'name': 'Hon Tony Abbott MP',
  'office': 1.0},
 {'For': 'Grayndler, New South Wales',
  'Party': 'Australian Labor Party',
  'address': '334A Marrickville Road\r\n                                             Marrickville, NSW, 2204',
  'link': '/Senators_and_Members/Parliamentarian?MPID=R36',
  'name': 'Hon Anthony Albanese MP',
  'office': 1.0},
 {'For': 'Bennelong, New South Wales',
  'Party': 'Liberal Party of Australia',
  'address': 'Suite 1, 44 - 46 Oxford St\r\n                                             Epping, NSW, 2121',
  'link': '/Senators_and_Members/Parliamentarian?MPID=M3M',
  'name': 'Mr John Alexander OAM, MP',
  'office': 1.0},
 {'For': 'Cowan, Western Australia',
  'Party': 'Australian Labor Party',
  'address': 'Shop 3, K

In [56]:
import pandas as pd

In [57]:
df = pd.DataFrame(offices)
df.head()

,For,Party,Title(s),address,link,name,office
0,"Warringah, New South Wales",Liberal Party of Australia,NaN,"Level 2, 17 Sydney Road\r\n ...",/Senators_and_Members/Parliamentarian?MPID=EZ5,Hon Tony Abbott MP,1.0
1,"Grayndler, New South Wales",Australian Labor Party,NaN,334A Marrickville Road\r\n ...,/Senators_and_Members/Parliamentarian?MPID=R36,Hon Anthony Albanese MP,1.0
2,"Bennelong, New South Wales",Liberal Party of Australia,NaN,"Suite 1, 44 - 46 Oxford St\r\n ...",/Senators_and_Members/Parliamentarian?MPID=M3M,"Mr John Alexander OAM, MP",1.0
3,"Cowan, Western Australia",Australian Labor Party,NaN,"Shop 3, Kingsway Shopping Centre\r\n ...",/Senators_and_Members/Parliamentarian?MPID=13050,Dr Anne Aly MP,1.0
4,"McPherson, Queensland",Liberal Party of Australia,Assistant Minister for Vocational Education an...,Ground Floor The Point 47 Watts Drive\r\n ...,/Senators_and_Members/Parliamentarian?MPID=230886,Hon Karen Andrews MP,1.0


In [63]:
df.groupby(['name','Party']).size().sort_values()

name                       Party                     
Dr Anne Aly MP             Australian Labor Party        1
Mr Ross Vasta MP           Liberal Party of Australia    1
Mr Ross Hart MP            Australian Labor Party        1
Mr Rob Mitchell MP         Australian Labor Party        1
Mr Peter Khalil MP         Australian Labor Party        1
Mr Pat Conroy MP           Australian Labor Party        1
Mr Nick Champion MP        Australian Labor Party        1
Mr Milton Dick MP          Australian Labor Party        1
Mr Michael Sukkar MP       Liberal Party of Australia    1
Mr Matt Keogh MP           Australian Labor Party        1
Mr Luke Howarth MP         Liberal Party of Australia    1
Mr Luke Gosling OAM, MP    Australian Labor Party        1
Mr Llew O'Brien MP         The Nationals                 1
Mr Kevin Hogan MP          The Nationals                 1
Mr Julian Leeser MP        Liberal Party of Australia    1
Mr Julian Hill MP          Australian Labor Party        1
Mr

In [64]:
df.to_csv('offices.csv')